In [1]:
# Packages

import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as f

import numpy as np
import os
import cv2
import json
print('OpenCV - version: ',cv2.__version__)
import pandas as pd

OpenCV - version:  4.0.0


### Key Biometric Ratio's 

In [2]:
# height in cm
persons_height = 177

# Toe to hip height as % of total height
toe_hip_height_ratio = 0.47

In [3]:
def get_vid_properties():
    #cap = cv2.VideoCapture('examples\media\mike_front_view_squat_trim.mp4')
    cap = cv2.VideoCapture('examples\media\mike_side_squat.mp4')
    width = int(cap.get(3))  # float
    height = int(cap.get(4)) # float
    cap.release()
    return width,height

In [4]:
print('Video Dimensions: ',get_vid_properties())

Video Dimensions:  (1080, 1920)


In [213]:
# Load keypoint data from JSON output
    
column_names = ['x', 'y', 'acc']

# Paths
#path_to_json = "output/squat_front/"
path_to_json = "output/squat_side_mike/"

# Import Json files
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
print('Found: ',len(json_files),'json keypoint frame files')
count = 0

width,height = get_vid_properties()

# instanciate dataframes 
body_keypoints_df = pd.DataFrame()
left_knee_df = pd.DataFrame()
right_knee_df = pd.DataFrame()
knee_sep_df = pd.DataFrame()
mid_hip_df = pd.DataFrame()
left_leg_df = pd.DataFrame()
right_leg_df = pd.DataFrame()


right_eye_df = pd.DataFrame()
right_elbow_df = pd.DataFrame()

r_big_toe_df = pd.DataFrame()
r_heel_df = pd.DataFrame()
r_hip_df = pd.DataFrame()
r_ankle_df = pd.DataFrame()
print('json files: ',json_files[0])   

# Right 9, 10, 11, 22
# Left 12, 13, 14, 19

#Loop through all json files in output directory
for file in json_files:
 #   try:
    temp_df = json.load(open(path_to_json+file))
    #print(temp_df)
    #print('Learning!!\n')
    #print(temp_df['part_candidates'][0])
    
    temp = []
    for k,v in temp_df['part_candidates'][0].items():
        
        # Single point detected
        if len(v) < 4:
            temp.append(v)
            #print('Extracted highest confidence points: ',v)
            
        # Multiple points detected
        elif len(v) > 4: 
            near_middle = width
            np_v = np.array(v)
            
            # Reshape to x,y,confidence
            np_v_reshape = np_v.reshape(int(len(np_v)/3),3)
            
            # compare x values
            for pt in np_v_reshape:
                if(np.absolute(pt[0]-width/2)<near_middle):
                    near_middle = np.absolute(pt[0]-width/2)
                    np_v_temp = list(pt)
                    #print('new method: ',np_v_temp)
         
            temp.append(np_v_temp)
            #print('Extracted highest confidence points: ',v[index_highest_confidence-2:index_highest_confidence+1])
        else:
            # No detection - record zeros
            temp.append([0,0,0])
            
    temp_df = pd.DataFrame(temp)
    temp_df = temp_df.fillna(0)
    #print(temp_df)

    try:
        prev_temp_df = temp_df
        body_keypoints_df= body_keypoints_df.append(temp_df)
        left_knee_df = left_knee_df.append(temp_df.iloc[13].astype(int))
        right_knee_df = right_knee_df.append(temp_df.iloc[10].astype(int))
        mid_hip_df = mid_hip_df.append(temp_df.iloc[8].astype(int))
        knee_sep_df = knee_sep_df.append(temp_df.iloc[13]-temp_df.iloc[10], ignore_index=True)
        left_leg_df = left_leg_df.append(temp_df.iloc[[12,13,14,19],:].astype(int))
        right_leg_df = right_leg_df.append(temp_df.iloc[[9,10,11,22],:].astype(int))
        right_eye_df = right_eye_df.append(temp_df.iloc[15].astype(int))
        right_elbow_df = right_elbow_df.append(temp_df.iloc[3].astype(int))
        r_big_toe_df = r_big_toe_df.append(temp_df.iloc[22].astype(int))
        r_heel_df = r_heel_df.append(temp_df.iloc[24].astype(int))
        
        r_hip_df = r_hip_df.append(temp_df.iloc[9].astype(int))
        r_ankle_df = r_ankle_df.append(temp_df.iloc[11].astype(int))
    except:
        print('bad point set at: ', file)
        
body_keypoints_df.head()
body_keypoints_df.columns = column_names
left_knee_df.columns = column_names
right_knee_df.columns = column_names
mid_hip_df.columns = column_names
left_leg_df.columns = column_names
right_leg_df.columns = column_names
right_eye_df.columns = column_names
right_elbow_df.columns = column_names

r_big_toe_df.columns = column_names
r_heel_df.columns = column_names
r_hip_df.columns = column_names
r_ankle_df.columns = column_names

body_keypoints_df.reset_index()
right_knee_df = right_knee_df.reset_index(drop = True)
left_knee_df = left_knee_df.reset_index(drop = True)
mid_hip_df = mid_hip_df.reset_index(drop = True)
left_leg_df = left_leg_df.reset_index(drop = True)
right_eye_df = right_eye_df.reset_index(drop = True)
right_elbow_df = right_elbow_df.reset_index(drop = True)

r_big_toe_df = r_big_toe_df.reset_index(drop = True)
r_heel_df = r_heel_df.reset_index(drop = True)
r_hip_df = r_hip_df.reset_index(drop = True)
r_ankle_df = r_ankle_df.reset_index(drop = True)

print('length of merged keypoint set: ',body_keypoints_df.size)


print(left_knee_df.head())
print(right_eye_df.head())

Found:  307 json keypoint frame files
json files:  mike_side_squat_000000000000_keypoints.json
length of merged keypoint set:  23025
       x       y  acc
0  494.0  1013.0  0.0
1  500.0  1013.0  0.0
2  500.0  1013.0  0.0
3  500.0  1013.0  0.0
4  500.0  1013.0  0.0
       x      y  acc
0  693.0  404.0  0.0
1  693.0  404.0  0.0
2  693.0  404.0  0.0
3  693.0  404.0  0.0
4  699.0  404.0  0.0


In [214]:
# Calculate motion flags (ascending vs decending)
def calc_y_gradients(points):
    points['gradient'] = 0
    grad_calc = []
    cur = 5
    prev = 5
    
    for pt in range(0,len(points)):
        if pt > 10:
            # subtract current y coordinate from previous y coordinate
            # positive values are moving down in the video (bottom of screen is larger number)
            # negative points moving up
      
            grad = np.mean(points['y'].values[pt-cur:pt]) - np.mean(points['y'].values[pt-(cur+prev):pt-cur])
            #print('Grad num: ',grad)
            
            if grad > 0:
                grad_calc.append(1)
            elif grad < 0:
                grad_calc.append(-1)
            else:
                grad_calc.append(0)
        else:
            grad_calc.append(0)
        # Now we need to store the gradient data in the points dataframe
    print('length of grad calc, ', len(grad_calc))
    print('length of points, ', len(points))
    points['gradient'] = grad_calc
    
    # return updated points set with gradient column
    return points

In [215]:
def calc_pt_delta(points, cntr_y, cntr_x):
    points['delta'] = 0
    delta_calc = []
    delta_previous = 0
    delta_previous_x = 0
    delta_previous_y = 0
    # scalars to set windows of frames to average position
    cur = 5
    prev = 10
    
    for pt in range(0, len(points)):
        # discard the first set of points (arbitrary, assuming consistent motion is not
        # occuring in opening few frames)
        if pt > 20:
            # calculate the delta between the current position and 
            # a reference point based on the person's starting position
            delta_x = np.abs(np.mean(points['x'].values[pt-cur:pt]) - cntr_x)
            delta_y = np.abs(np.mean(points['y'].values[pt-cur:pt]) - cntr_y)
            delta_previous_x = np.abs( np.mean(points['x'].values[pt-(cur+prev):pt-cur])- cntr_x)
            delta_previous_y = np.abs( np.mean(points['y'].values[pt-(cur+prev):pt-cur]) - cntr_y)
            delta = int(np.sqrt(delta_x*delta_x + delta_y*delta_y))
            delta_previous = int(np.sqrt(delta_previous_x*delta_previous_x +
                                         delta_previous_y*delta_previous_y))
            
            if delta > delta_previous+2:
                delta_calc.append(1)
                delta_previous = delta
            elif delta < delta_previous-2:
                delta_calc.append(-1)
                delta_previous = delta
            else:
                delta_calc.append(0)
        else:
            delta_calc.append(0)
                
    print('Length of delta calc: ', len(delta_calc))
    points['delta'] = delta_calc
    
    return points
                

In [216]:
# create connected poly ID
def poly_id(points):
    points['polyid'] = 0
    id_var = 'gradient'
    #id_var = 'delta'
    grad_status = 0
    delta_status = 0
    id_num = 0
    poly_calc = []
    
    for pt in range(0,len(points)):
       
        if grad_status == points[id_var].values[pt]:
            poly_calc.append(id_num)
        else:
            grad_status = points[id_var].values[pt]
            id_num += 1
            poly_calc.append(id_num)
            
    points['polyid'] = poly_calc
    
    return points

In [217]:
# create connected poly ID
def delta_id(points):
    points['delta_id'] = 0
    #id_var = 'gradient'
    id_var = 'delta'
    grad_status = 0
    delta_status = 0
    id_num = 0
    poly_calc = []
    
    for pt in range(0,len(points)):
       
        if grad_status == points[id_var].values[pt]:
            poly_calc.append(id_num)
        else:
            grad_status = points[id_var].values[pt]
            id_num += 1
            poly_calc.append(id_num)
            
    points['delta_id'] = poly_calc
    
    return points

In [218]:
# create connected poly ID
def create_pos_id(points, id_var = 'position'):
    col_name = id_var+'_id'
    points[col_name] = 0
    #id_var = 'gradient'
    status = 0
    id_num = 0
    poly_calc = []
    
    for pt in range(0,len(points)):
       
        if status == points[id_var].values[pt]:
            poly_calc.append(id_num)
        else:
            status = points[id_var].values[pt]
            id_num += 1
            poly_calc.append(id_num)
            
    points[col_name] = poly_calc
    
    return points

In [219]:
# Calculate motion flags (ascending vs decending)
def motion(pts):
    column_name = 'motion'
    pts[column_name] = 0
    motion_calc = []
    seg_num = max(pts['position_id'].values)
    print('seg num ', seg_num)
    max_y = 0
    seg = 0
    for seg in range(0,seg_num):
        
        seg_pts = pts[pts['position_id']==seg]
        if len(seg_pts) > 0:
            print('Number of segment points: ',len(seg_pts))
            max_y = max(seg_pts['y'].values)
            min_y = min(seg_pts['y'].values)
            
            #print(pts.loc[(pts["position_id"] == seg) & (pts["y"] == max_y), "motion"])
            
            # Check position values for given segment
            # Position < 0 means we're at the bottom of te motion
            # Bottom of the motion means we'll be finding the point at which we start to move up
            # to find the bottom of the range of motion we look for the largest y value
            # we'll assign '1' to that point because we'll be moving up after that point.
            
            if seg_pts['position'].values[0] < 0:
                query_motion = pts.query('position_id == @seg & y == @max_y').index
                pts.at[query_motion[0], 'motion'] = 1
            
            elif seg_pts['position'].values[0] > 0:
                query_motion = pts.query('position_id == @seg & y == @min_y').index
                pts.at[query_motion[0], 'motion'] = -1


In [220]:
# create connected poly ID
def motion_id(points):
    column_name = 'motion_id'
    motion_calc = []
    # higher y values correspond to the lower part of the image
    # standing upright ~ lower y coordinate value
    
    last = 0
    direction = 'down'
    cur = 0
    
    for pt in range(0,len(points)):
        cur = points['motion'].values[pt]
        
        if cur > 0:
            direction = 'up'
            motion_calc.append(direction)
        elif cur < 0:
            direction = 'down'
            motion_calc.append(direction)
        else:
            motion_calc.append(direction)
            
    # Now we need to store the position data in the points dataframe
    points[column_name] = motion_calc
    
    print(points.head())
    
    # return updated points set with gradient column
    return points

In [221]:
# Calculate position flags (upper vs lower)
# current position is > than mid point ~ assign -1 --> indicates bottom half of motion range
# current position is < than mid point ~ assign 1 --> indicates top half of motion range

def position(points):
    column_name = 'position'
    points[column_name] = 0
    motion_calc = []
    # higher y values correspond to the lower part of the image
    # standing upright ~ lower y coordinate value
    pos_min = np.max(points['y'].values)
    pos_max = np.min(points['y'].values)

    mid_pt = int((pos_min+pos_max) / 2)
    cur = 0
    
    for pt in range(0,len(points)):
        cur = points['y'].values[pt]
        if cur > mid_pt:
            motion_calc.append(-1)
        elif cur < mid_pt:
            motion_calc.append(1)
        else:
            motion_calc.append(0)
            
    # Now we need to store the position data in the points dataframe
    points[column_name] = motion_calc
    
    # return updated points set with gradient column
    return points

In [222]:
# create connected poly ID
def motion_id_seg(points, id_var = 'motion_id'):
    col_name = id_var+'_seg'
    points[col_name] = 0
    #id_var = 'gradient'
    status = 0
    id_num = 0
    poly_calc = []
    
    for pt in range(0,len(points)):
       
        if status == points[id_var].values[pt]:
            poly_calc.append(id_num)
        else:
            status = points[id_var].values[pt]
            id_num += 1
            poly_calc.append(id_num)
            
    points[col_name] = poly_calc
    
    return points

In [223]:
def calc_knee_disp(pts_l,pts_r, cntr_y, cntr_x):
    pts_l['knee_delta'] = 0
    pts_r['knee_delta'] = 0
    pts_l['knee_flag'] = 0
    pts_r['knee_flag'] = 0
    delta_r_flags = []
    delta_l_flags = []
    delta_calc = []

    for pt in range(0, len(pts_r)):
        # calculate the delta between the current position and 
        # a reference point based on the person's starting position
        mid_x = int((pts_r['x'].values[pt] + pts_l['x'].values[pt])/2)
        delta = mid_x - cntr_x
        delta_calc.append(delta)
                
    print('Length of delta calc: ', len(delta_calc))
    pts_l['knee_delta'] = delta_calc
    pts_r['knee_delta'] = delta_calc
    
    knee_delta_mean = np.mean(np.absolute(pts_r['knee_delta'].values))
    
    for pt in range(0, len(pts_r)):
        if np.absolute(pts_l['knee_delta'].values[pt]) > (2.5*knee_delta_mean):
            if pts_l['knee_delta'].values[pt] > 0:
                delta_r_flags.append(1)
                delta_l_flags.append(0)
                
            else:
                delta_l_flags.append(1)
                delta_r_flags.append(0)
        else:
            delta_l_flags.append(0)
            delta_r_flags.append(0)
    print('len of delta_l_flags ', len(delta_l_flags))
    print('len of delta_r_flags ', len(delta_r_flags))
    print('length of pts_l', len(pts_l))
    pts_l['knee_flag'] = delta_l_flags
    pts_r['knee_flag'] = delta_r_flags
    
    return pts_l, pts_r

In [224]:
# Calculate gradients for points sets.

left_knee_df = calc_y_gradients(left_knee_df)
right_knee_df = calc_y_gradients(right_knee_df)
mid_hip_df = calc_y_gradients(mid_hip_df)

cntr_y = int((left_knee_df['y'].values[0] + right_knee_df['y'].values[0]) / 2)
cntr_x = int((left_knee_df['x'].values[0] + right_knee_df['x'].values[0]) / 2)

left_knee_df = calc_pt_delta(left_knee_df, cntr_y, cntr_x)
rigth_knee_df = calc_pt_delta(right_knee_df, cntr_y, cntr_x)

# Knee out of position flag section
left_knee_df, right_knee_df = calc_knee_disp(left_knee_df, right_knee_df, cntr_y, cntr_x)

# Direction of motion / rep calculation
mid_hip_df = position(mid_hip_df)
mid_hip_df = create_pos_id(mid_hip_df, 'position')
motion(mid_hip_df)
mid_hip_df = motion_id(mid_hip_df)
mid_hip_df = motion_id_seg(mid_hip_df)

# assigning motion id's to left/right knee
left_knee_df['position_id'] = mid_hip_df['position_id'].values
right_knee_df['position_id'] = mid_hip_df['position_id'].values

left_knee_df['motion_id_seg'] = mid_hip_df['motion_id_seg'].values
right_knee_df['motion_id_seg'] = mid_hip_df['motion_id_seg'].values

left_knee_df['motion_id'] = mid_hip_df['motion_id'].values
right_knee_df['motion_id'] = mid_hip_df['motion_id'].values



left_knee_df = poly_id(left_knee_df)
right_knee_df = poly_id(right_knee_df)
mid_hip_df = poly_id(mid_hip_df)



'''
print('right number of pos grad',  len(right_knee_df[right_knee_df.iloc[:,3]>0]))
print('right number of neg grad',  len(right_knee_df[right_knee_df.iloc[:,3]<0]))

#print(right_knee_df[right_knee_df.iloc[:,3]!=0])
#print(right_knee_df[0:20])
print('right num of neg delta',  len(right_knee_df[right_knee_df['delta']<0]))
print('right num of pos delta',  len(right_knee_df[right_knee_df['delta']>0]))
'''
left_knee_df = delta_id(left_knee_df)
right_knee_df = delta_id(right_knee_df)

print('\n# of Segment ids: ')
print('Right Knee Seg ID Gradient: ', right_knee_df['polyid'].nunique())
print('Right Knee Seg ID Deltas: ', right_knee_df['delta_id'].nunique())
print('Mid Hip Seg ID Gradient: ', mid_hip_df['polyid'].nunique())
print('Mid Hip Seg ID Position_id: ', mid_hip_df['position_id'].nunique())
print('Ok great, now to correctly identify the top and bottom of the motion')
mid_hip_df.reset_index()
print(mid_hip_df.head())

length of grad calc,  307
length of points,  307
length of grad calc,  307
length of points,  307
length of grad calc,  307
length of points,  307
Length of delta calc:  307
Length of delta calc:  307
Length of delta calc:  307
len of delta_l_flags  307
len of delta_r_flags  307
length of pts_l 307
seg num  7
Number of segment points:  75
Number of segment points:  32
Number of segment points:  54
Number of segment points:  31
Number of segment points:  48
Number of segment points:  34
       x      y  acc  gradient  position  position_id  motion motion_id
0  500.0  790.0  0.0         0         1            1       0      down
1  500.0  795.0  0.0         0         1            1       0      down
2  500.0  790.0  0.0         0         1            1       0      down
3  506.0  790.0  0.0         0         1            1       0      down
4  501.0  790.0  0.0         0         1            1       0      down

# of Segment ids: 
Right Knee Seg ID Gradient:  48
Right Knee Seg ID Deltas:

### Plan to Completion

#### End state:
1. Continuous color segment for down and up motion of the rep

#### Play it out:
1. Person will start in the same position they finish
2. Each Rep should consist of only 2 segments
3. The raw gradient, lightly processed gradient and segment IDs are insufficient
4. Likely need to write custom motion processing logic and or filtering functions

#### Motion Processing
1. A rep will start at the max or within max +- 10%
    - Max can be determined by sampling t0 or max
    - Min can be sampled by taking the min of the set? (for test purposes yes, reality no - failed attempts)
2. <b>Possible steps to measure</b>
    - Determine max and min based on ranges within total set
    - First Pass: determine areas within 50% of min or max?
        - Assign labels as upper and lower
        - Assign <b>Position Segment IDs</b> based on upper and lower
    - Second Pass: identify the local max and min within each segment
        - Assign <b>Motion Segment IDs</b> based on min max within each position segment
        - For a given position segment id
        - determine the max - set to -1 (down) (fisrt occurence)
        - determine the min - set to 1 (up) (first occurence)
            - challenge will be to correctly assign back to original df?
            - all other values 
        - 

#### Filtering Funcs
1. Smoothing
- Total number of segments should be equal to 2x reps
- Rep is measured as starting from max - returning to max
    

### Plot raw points at every graident change

In [225]:
# Plotly Plot

### TO DO - sub set the plot where gradient not equal zero, add transparency to points, should be good.

import plotly
plotly.tools.set_credentials_file(username='aduxbury', api_key='1vW1xxY8a14YJ6cd5Efw')
trace0 = go.Scatter(
    x = left_knee_df.loc[left_knee_df['gradient'] != 0, 'x'],
    y = left_knee_df.loc[left_knee_df['gradient'] != 0, 'y'],
    mode = 'markers',
    name = 'Left Knee',
    marker=dict(
        size=8,
        color = left_knee_df.loc[left_knee_df['gradient'] != 0, 'gradient'], #set color equal to a variable
        colorscale='RdBu',
        showscale=True
    )
)

trace1 = go.Scatter(
    x = right_knee_df.loc[right_knee_df['gradient'] != 0, 'x'],
    y = right_knee_df.loc[right_knee_df['gradient'] != 0, 'y'],
    mode = 'markers',
    name = 'Right Knee',
    marker=dict(
        size=8,
        color = right_knee_df.loc[right_knee_df['gradient'] != 0, 'gradient'], #set color equal to a variable
        colorscale='RdBu',
        showscale=True),
)


trace2 = go.Scatter(
    x = mid_hip_df.loc[mid_hip_df['gradient'] != 0, 'x'],
    y = mid_hip_df.loc[mid_hip_df['gradient'] != 0, 'y'],
    mode = 'markers',
    name = 'Mid Hip',
    marker=dict(
        size=8,
        color = mid_hip_df.loc[mid_hip_df['gradient'] != 0, 'gradient'], #set color equal to a variable
        colorscale='RdBu',
        showscale=True),
)

x = right_knee_df.loc[right_knee_df['gradient'] != 0, 'x']
y = right_knee_df.loc[right_knee_df['gradient'] != 0, 'y']
print('length of x ', len(x))
print('length of y ', len(y))


layout = go.Layout(
    yaxis=dict(autorange='reversed'))
    
data = [trace0, trace1, trace2]
fig = go.Figure(data=data, layout=layout)


py.iplot(fig, filename = 'front_squat')

length of x  274
length of y  274


In [226]:
# Plotly Plot

### TO DO - sub set the plot where gradient not equal zero, add transparency to points, should be good.
plot_var = 'delta'
import plotly
plotly.tools.set_credentials_file(username='aduxbury', api_key='1vW1xxY8a14YJ6cd5Efw')
trace0 = go.Scatter(
    x = left_knee_df.loc[left_knee_df[plot_var] != 0, 'x'],
    y = left_knee_df.loc[left_knee_df[plot_var] != 0, 'y'],
    mode = 'markers',
    name = 'Left Knee',
    marker=dict(
        size=8,
        color = left_knee_df.loc[left_knee_df[plot_var] != 0, plot_var], #set color equal to a variable
        colorscale='RdBu',
        showscale=True
    )
)

trace1 = go.Scatter(
    x = right_knee_df.loc[right_knee_df[plot_var] != 0, 'x'],
    y = right_knee_df.loc[right_knee_df[plot_var] != 0, 'y'],
    mode = 'markers',
    name = 'Right Knee',
    marker=dict(
        size=8,
        color = right_knee_df.loc[right_knee_df[plot_var] != 0, plot_var], #set color equal to a variable
        colorscale='RdBu',
        showscale=True),
)
x = right_knee_df.loc[right_knee_df[plot_var] != 0, 'x']
y = right_knee_df.loc[right_knee_df[plot_var] != 0, 'y']


print('length of x ', len(x))
print('length of y ', len(y))
layout = go.Layout(
    yaxis=dict(autorange='reversed'))
    
data = [trace0, trace1]
fig = go.Figure(data=data, layout=layout)


py.iplot(fig, filename = 'front_squat')

length of x  224
length of y  224


### plot histragram of segment 'lengths'
Ideally there should be 1 seg down, 1 seg up for each 'rep' in the exercise, but due to sampling that does not occur
Gradient changes occur when points are sampled at different grid position, graident changes truncate segments
Back and forth changes in gradients create an abundance of short segments (~50 segments with fewer than 10 points)
#### Now that a graph has been made, we've #datascienced so, this work is legit.
What we can do (hopefully) is drop every segment with fewer than 10 points.

In [227]:

import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np

x = right_knee_df['delta_id'].value_counts()
print('Max length of seg: ',np.max(x))
data = [go.Histogram(x=x)]

py.iplot(data, filename='basic histogram')

Max length of seg:  27


In [228]:

import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np

x = right_knee_df['knee_delta'].values
print('Max length of seg: ',np.max(x))
print('Min length of seg: ',np.min(x))
print('Mean abs knee delta: ', np.mean(np.absolute(x)))
print('Mean knee delta: ', np.mean(x))

data = [go.Histogram(x=x)]

py.iplot(data, filename='basic histogram')

Max length of seg:  58
Min length of seg:  -36
Mean abs knee delta:  20.895765472312704
Mean knee delta:  7.175895765472313


In [229]:
# QC point sets against number of frames
print(knee_sep_df.shape)
print(left_knee_df.shape)
print(right_knee_df.shape)

(307, 3)
(307, 12)
(307, 12)


In [230]:
# Func to place text on an image
def draw_label(img, text, color_select=(255,255,255)):
    font_face = cv2.FONT_HERSHEY_SIMPLEX
    scale = 0.8
    bg_color = (0,0,0)
    thickness = cv2.FILLED
    margin = 10
    
    # image dimensions
    img_y = img.shape[0]
    img_x = img.shape[1]

    txt_size = cv2.getTextSize(text, font_face, scale, thickness)

    # Set text print position to lower middle of screen
    # This takes the image size and text size, then positions the message centered
    pos = (int(img_y*0.98), (int(img_x/2) - int((txt_size[0][0])/2)))
    
    # reverses y,x order for plotting as (x,y)
    pos = pos[::-1]
    
    # define end points for text box
    # This is used for printing a bounding box
    end_x = pos[0] + txt_size[0][0] + margin
    end_y = pos[1] - txt_size[0][1] - margin
    
    # background rectangle
    #cv2.rectangle(img, (pos[0]-margin,pos[1]+margin), (end_x, end_y), bg_color, thickness)
    # text
    cv2.putText(img, text, pos, font_face, scale, color_select, 2)


In [231]:
# Func to place text on an image
#Circle(img, center, radius, color, thickness=1, lineType=8, shift=0)
def draw_point(img, point, color_select = (255, 255, 255)):
    scale = 0.8
    bg_color = (0,0,0)
    thickness = 4
    radius = 4

    pos = (int(point[0]), int(point[1]))
    
    cv2.circle(img, pos, radius, color_select, thickness)

In [232]:
def draw_poly_line(img, pts, color_select = (255,255,255), thick = 2):
    poly_line_thickness = thick
    poly_closed = False
    pts = pts[:,0:2]
    pts = pts.reshape((-1,1,2))
    
    cv2.polylines(img, np.int32([pts]), poly_closed, color_select, thickness=poly_line_thickness)


In [233]:
# take point data grouped by segments and plot individual poly segments

def draw_line_set(img, pts, color_select = (220,220,200)):

    # Figure out the number of segments
    seg_num = np.max(pts['motion_id_seg'].values)
    red = (10, 230, 10)
    blue = (255, 255, 255)
    previous_seg_color = blue

    counter = 0
    counter2 = 0
    seg = 1
    #print('length of pts: ', len(pts), ' seg_num, ', seg_num)
    
    # Plot each segment separately and with it's gradient appropriate color
    while seg < seg_num+1:
        
        seg_pts = pts[pts['motion_id_seg']==seg]
        if len(seg_pts) > 0:
            # based on histogram of points - we want to drop all short points from plotting
            #print('inside for loop: ', len(seg_pts))
            #print('Seg ', seg)
            if(seg_pts['motion_id'].values[0] == 'up'):
                # draw a red line for up
                draw_poly_line(img, seg_pts.values, red)
                counter = counter+ 1
                previous_seg_color = blue
                #print('Total pass through up: ',counter)

            elif(seg_pts['motion_id'].values[0] == 'down'):
                # draw a blue line for down
                draw_poly_line(img, seg_pts.values, )
                previous_seg_color = red
                counter2 = counter2 + 1 
                #print('length of pts ',len(seg_pts.values))

            else:
                draw_poly_line(img, seg_pts.values, previous_seg_color)
        
        seg+=1

In [234]:
'''
np.max(left_knee_df['polyid'].values)
seg_pts = right_knee_df[right_knee_df['motion_id_seg']<3]
print('counting ', seg_pts['motion_id'].values[0]=='down')
print('counting ', seg_pts.iloc[0:20,6:8])
'''

"\nnp.max(left_knee_df['polyid'].values)\nseg_pts = right_knee_df[right_knee_df['motion_id_seg']<3]\nprint('counting ', seg_pts['motion_id'].values[0]=='down')\nprint('counting ', seg_pts.iloc[0:20,6:8])\n"

### Plan to sexy MVP

1. Add grey fade + alpha channels for plotting
    - want to call point - 90 points primary
    - zero to 90 as grey with incremental alpha (place in fnc call)
    - White circle on knee point
#### 2. Develop flags for knee movements out of alignment
    - calc Original (standing) center knees (x-coord) (a)
    - for each pair of knee points - calc mid point (x-coord) (b)
    - calculate the displacement (+-) from the current (b) to center knees (a)
    - Calculate a flag based on % displacement from center (a)
    - When the flag is triggered - change display text and corresponding knee color
3. Add message for knee out of position
#### 4. Calculate and dispay velocity
    - Determine pixes to distance relationship x pixes = 1m
        - We'll have a height measure m and a pixel value ~ pixels / m
    - Determine the time
        - Length of the input vid in seconds, number of frames ~ frames / second
    - Calculate and display values
        - Current velocity = average velocity over the past quarter second, udated every quarter second
        - Average decent velocity = total distance for all down flagged motions, and the total frame count
        - Average Ascent velocity = total distance for all up flags, and the total frame count
        
#### 5. Clean inputs (discard background points)
    - filter points based on location

#### 6. Determine view angle (side vs front)
    - Refactor plotting to handle either scenario (make a func call)
#### 7. Side view key factors
    - 1 is bar path
    - 2 Squat depth check (y pos of hip <= y pos knee)
5. Clean up code base
6. Build tooling to load video in openPose, extract point data, process point data and plot new video

In [235]:
def check_form(frame, pts_l, pts_r, msg = "Good Form"):
    
    if pts_l['knee_flag'] == 1:
        msg = "Form Break! - Left Knee"
        draw_point(frame,pts_l,(0,0,255))
        draw_label(frame,msg,(0,0,255))
        
    elif pts_r['knee_flag'] == 1:
        msg = "Form Break! - Right Knee"
        draw_point(frame,pts_r,(0,0,255))
        draw_label(frame,msg,(0,0,255))
    else:
        draw_label(frame,msg,(255,255,255))
    return msg
    
    

In [236]:
def get_pixel_per_meter():
    # subtract toe y position from hip y position
    # relate pixel distance to peron's height / 2
    pixels = right_leg_df['y'].iloc[3] - right_leg_df['y'].iloc[0]
    meters = (persons_height*toe_hip_height_ratio)/100
    pix_per_meter = pixels/meters
   
    return pix_per_meter

In [237]:
def get_distance(pts):
    pixels_per_meter = get_pixel_per_meter()
    distance = 0
    total_dist_pix = []
    for pt in range(1,len(pts)):
        total_dist_pix.append(np.abs(pts['y'].iloc[pt]-pts['y'].iloc[pt-1]))
    
    total_dist_pix = np.sum(total_dist_pix)
    # expected distance in meters
    distance = total_dist_pix / pixels_per_meter
    # set precision 2 decimals
    distance = int(distance * 100)/100
    
    return distance


In [238]:
def get_velocity(pts, frame_fps):
    # time in seconds
    time = len(pts)/frame_fps
    distance = get_distance(pts)
    velocity = distance / time
    # convert to 2 decimals
    velocity = int(velocity*100)/100
    
    return velocity
    

In [239]:
def get_average_velocity(pts, frame_fps):
    
    seg_num = np.max(pts['motion_id_seg'].values)
    
    avrg_asc_vel = [0]
    avrg_dec_vel = [0]
    
    for seg in range(1,seg_num+1):
        if len(pts[pts['motion_id_seg']==seg]) > 2:
            vel = get_velocity(pts[pts['motion_id_seg']==seg], frame_fps)
        else:
            vel = 0
        if seg%2 == 1:
            avrg_dec_vel.append(vel)
        else:
            avrg_asc_vel.append(vel)
    
    avrg_asc_vel = int(np.average(avrg_asc_vel)*100)/100
    avrg_dec_vel = int(np.average(avrg_dec_vel)*100)/100
    
    
    return avrg_asc_vel, avrg_dec_vel

In [240]:
def get_rep(pts, count):
    if pts['motion_id_seg'].iloc[count] == 1:
        rep = 1
    elif pts['motion_id_seg'].iloc[count] % 2 == 0:
        rep = int(pts['motion_id_seg'].iloc[count]/2)
    else:
        rep = int(np.round(pts['motion_id_seg'].iloc[count]/2+0.1))
    return rep

In [279]:
def draw_stats_box(frame, velocity, avrg_ascent, avrg_decent,angle, count):

    title_text = "MyStrengthBook.com"
    
    rep = get_rep(mid_hip_df, count)
        
    cur_rep = "Rep: "+str(rep)
    cur_frame = "Frame: "+str(count)
    vel_text = "Movement Speed: "+str(velocity)+" m/s"
    avrg_dec_vel = "Avrg Decent Speed: "+str(avrg_decent)+" m/s"
    avrg_asc_vel = "Avrg Ascent Speed: "+str(avrg_ascent)+" m/s"
    angle_text = "Knee Angle: "+str(angle)+" deg"
    if angle < 92:
        angle_text = "Knee Angle: "+str(angle)+" deg, Good."
    
    
    # string used to set text box size
    for_size_text = "Avrg Ascent Speed: "+str(10.00)+" m/s"
    
    box_text = [cur_rep, cur_frame, vel_text, avrg_dec_vel, avrg_asc_vel, angle_text]
    
    font_face = cv2.FONT_HERSHEY_DUPLEX
    scale = 0.8
    bg_color = (0,0,0)
    # MSB Green
    box_color = (74, 194, 108)
    # charcoal text
    text_color = (50, 50, 50)
    thickness = cv2.FILLED
    margin = 10
    
    # image dimensions
    #img_y = img.shape[0]
    #img_x = img.shape[1]
    
    # Determine size of text box
    text = for_size_text
    txt_size = cv2.getTextSize(text, font_face, scale, thickness)
    text_height = txt_size[0][1]
    text_width = txt_size[0][0]
    
    box_height = len(box_text)*text_height + len(box_text)*margin + margin*8
    box_width = text_width + 4*margin
    
    # Set text print position to lower middle of screen
    # This takes the image size and text size, then positions the message centered
    text = title_text
    txt_size = cv2.getTextSize(text, font_face, scale, thickness)
    txt_size = txt_size[0][0]
    title_pos = (int(box_width/2-txt_size/2),int(2*margin+text_height))
    
    # background rectangle
    cv2.rectangle(frame, (0,0), (5+box_width, 5+box_height), text_color, -1)
    cv2.rectangle(frame, (0,0), (box_width, box_height), box_color, -1)
    
    counter = 0
    y_pos = 3*text_height
    for txt in box_text:
        cv2.putText(frame, txt, (2*margin,3*margin+y_pos), font_face, scale, text_color, 1)
        counter = counter + 1
        y_pos = y_pos + text_height+int(margin)
    
    
    # Title Text
    cv2.putText(frame, title_text, title_pos, font_face, scale, text_color, 1)
    

In [242]:
def draw_bar_vert_line(frame, eye_pts, elbow_pts, color_select):
    pts = pd.DataFrame()
    pts = pts.append(eye_pts)
    pts = pts.append(eye_pts)
    pts['x'] = elbow_pts['x']
    pts.iloc[1]['y'] = height
    cols = ['x','y','acc']
    pts = pts[cols]
    draw_poly_line(frame, pts.values, color_select)

In [243]:
def draw_bar_box(frame, heel_pt, toe_pt, color_select):
    fudge_fact = 20
    heel_x = int(np.mean(heel_pt['x']))-15
    toe_x = int(np.mean(toe_pt['x']))+15
    top = 0
    base = height
    cv2.rectangle(frame, (heel_x,top), (toe_x, base), color_select, -1)
    

In [350]:
def draw_knee_line(frame, knee_pt, toe_pt, color_select):
    knee_x = int(np.mean(knee_pt['x']))+40
    knee_y = int(np.mean(knee_pt['y']))-100
    toe_y = int(toe_pt['y'])
    pts = np.array([[knee_x,toe_y], [knee_x,knee_y]])
    pts = pd.DataFrame(pts)
    cols = ['x','y']
    pts.columns = cols
    draw_poly_line(frame,pts.values,color_select,thick=3)
    #print('draw_knee')

In [351]:
# Calculate shin to thigh angle
import math
def get_knee_angle(pt1, pt2, pt3):
    
    # double result = atan2(P3.y - P1.y, P3.x - P1.x) -
                #atan2(P2.y - P1.y, P2.x - P1.x);
    #https://stackoverflow.com/questions/1211212/how-to-calculate-an-angle-from-three-points
    
    angle = math.atan2(pt3['y'] - pt1['y'], pt3['x'] - pt1['x']) - \
                math.atan2(pt2['y'] - pt1['y'], pt2['x'] - pt1['x'])
        
    angle = float(math.degrees(angle))
    angle = np.round(angle)
    angle = 360 - angle
    angle = int(angle)
    #print(angle)
    return angle

### Video with Knee Tracking Lines - Testing v2 up/down motion tracking

In [355]:
#cap = cv2.VideoCapture('examples\media\mike_front_view_squat_trim.mp4')
cap = cv2.VideoCapture('examples\media\mike_side_squat.mp4')
frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
frame_fps = cap.get(cv2.CAP_PROP_FPS)
# 4 samples per second ~ 1 every 250ms
sample_interval = 4
vel_inter = int(np.round(frame_fps/sample_interval))

# Status box values
velocity = 0
avrg_decent = 0
avrg_ascent = 0
angle =0
max_x = 0
cur_rep = 1

print("Number of Frames in Input Video: ",frame_count)
print("Frame Rate Input: ", frame_fps)

alpha = 0
text = "Current Frame: "
# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

width = int(cap.get(3))  # float
height = int(cap.get(4)) # float

cv2.namedWindow('Peach Factor',cv2.WINDOW_AUTOSIZE)

### Init output video
out = cv2.VideoWriter('peach_factory_v2.avi',cv2.VideoWriter_fourcc('M','J','P','G'),
                                  25, (width,height))
count = 0

while(True):
    
    ret, frame = cap.read()
    if ret == True:
        buffer = frame.copy()
        if count > 0:
            
            frame_msec = int(cap.get(cv2.CAP_PROP_POS_MSEC))
     
            #if count < 150:
                #right_temp_df = right_knee_df.iloc[0:count]
                #left_temp_df = left_knee_df.iloc[0:count]
                # Expected right knee, red line --> direct of motion is up
                #draw_line_set(frame, right_knee_df.iloc[0:count])
                # Expected left knee, red line --> direct of motion is up
                #draw_line_set(frame, left_knee_df.iloc[0:count])
            
            #else:
                #right_temp_df = right_knee_df.iloc[count-89:count]
                #left_temp_df = left_knee_df.iloc[count-89:count]
                #draw_line_set(frame, right_knee_df.iloc[count-149:count])
                # Expected left knee, red line --> direct of motion is up
                #draw_line_set(frame, left_knee_df.iloc[count-149:count])
                #alpha = 1-(count/frame_count)+0.3
                
                #draw_line_set(frame, right_knee_df.iloc[0:count-149])
                # Expected left knee, red line --> direct of motion is up
                #draw_line_set(buffer, left_knee_df.iloc[0:count-149])
            
            for point in range(0,4):
                #draw_point(frame, left_leg_df.iloc[(count*4)+(point)], (150,150,0))
                draw_point(frame, right_leg_df.iloc[(count*4)+(point)], (150,150,0))
                
            #draw_poly_line(buffer, left_leg_df.iloc[count*4:(count*4)+4].values, (255,255,255))
            draw_poly_line(frame, right_leg_df.iloc[count*4:(count*4)+4].values, (255,255,255))
            
            # draw bar path vertical path
            #draw_bar_vert_line(frame, right_eye_df.iloc[0], right_elbow_df.iloc[0], (0,255,255))
           
                    
            if count%int(vel_inter/4) == 0 and count > int(vel_inter/4):
                angle = get_knee_angle(right_knee_df.iloc[count],
                                       r_hip_df.iloc[count], r_ankle_df.iloc[count])
                
            if count > 10:
                # draw bar box
                draw_bar_box(buffer, r_heel_df.iloc[count-10:count], r_big_toe_df.iloc[count-10:count],(0,255,0) )
                # draw knee line
                if(angle < 120):
                    if right_knee_df.iloc[count]['x'] > max_x:
                        max_x = right_knee_df.iloc[count]['x']
                        max_count = count
                    draw_knee_line(frame, right_knee_df.iloc[max_count],
                                   r_big_toe_df.iloc[0], (255,0,0))
                    if get_rep(mid_hip_df,count) != cur_rep:
                        max_x = 0
                        max_count = count
                        cur_rep = get_rep(mid_hip_df,count)
            # draw bar point
            
            alpha = 0.2
            #draw_point(buffer, left_knee_df.iloc[count], (0,255,0))
            draw_point(buffer, right_knee_df.iloc[count], (0,255,0))
            
            # Make Velocity Calculation
            if count%vel_inter == 0 and count > vel_inter:
                velocity = get_velocity(mid_hip_df.iloc[(count-vel_inter):count], frame_fps)
                avrg_ascent, avrg_decent = get_average_velocity(mid_hip_df.iloc[0:count], frame_fps)
                

            #check_form(frame, left_knee_df.iloc[count], right_knee_df.iloc[count])
            
            #draw_label(frame, text+str(count)+", "+msg)
            #cv2.putText(frame,text,(150,500), font, 1,(255,255,255),2,cv2.LINE_AA)
            
            cv2.addWeighted(buffer, alpha, frame,1-alpha, 0, frame)
            
            draw_stats_box(frame,velocity, avrg_ascent, avrg_decent, angle, count)
            
            cv2.imshow('Peach Factor',frame)
            
            ### Write output video
            out.write(frame)
            
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break
    else:
        break
    count+=1
     
cap.release()
out.release()
cv2.destroyAllWindows()

Number of Frames in Input Video:  307.0
Frame Rate Input:  29.97002997002997


In [356]:
# Compress output AVI into clean and tiny MP4

from subprocess import check_output
check_output("del peach_factory_25fps.mp4", shell=True).decode()
check_output("ffmpeg -i peach_factory_v2.avi -vcodec h264 -acodec mp2 peach_factory_25fps.mp4", shell=True).decode()
# ffmpeg -i peach_factory_v2.avi -vcodec h264 -acodec mp2 peach_factory_v2.mp4

''

In [301]:
#mid_hip_df

### Video with Knee Points only

In [37]:
#cap = cv2.VideoCapture('examples\media\squat_front_ad_trim.mp4')
cap = cv2.VideoCapture('examples\media\mike_front_view_squat.mp4')
print("Number of frames in video: ",cap.get(cv2.CAP_PROP_FRAME_COUNT))
# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

cv2.namedWindow('Peach Factor',cv2.WINDOW_AUTOSIZE)
text = "Good Form!"

count = 0

while(True):
    
    ret, frame = cap.read()
    if ret == True:
        if count > 2:
              
            right_temp_df = right_knee_df.iloc[count]
            left_temp_df = left_knee_df.iloc[count]
            
            if right_temp_df['gradient']>0:
                draw_point(frame, right_temp_df, (255,0,0))
            elif right_temp_df['gradient']<0:
                draw_point(frame, right_temp_df, (0,0,255))
            if left_temp_df['gradient']>0:
                draw_point(frame, left_temp_df, (255,0,0))
            elif left_temp_df['gradient']<0:
                draw_point(frame, left_temp_df, (0,0,255))    
                
            #draw_point(frame, left_knee_df.iloc[count])
            draw_label(frame, text)
            #cv2.putText(frame,text,(150,500), font, 1,(255,255,255),2,cv2.LINE_AA)
            cv2.imshow('Peach Factor',frame)

            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
    else:
        break
    count+=1
     
cap.release()
cv2.destroyAllWindows()

Number of frames in video:  0.0
Error opening video stream or file


In [38]:
#cap = cv2.VideoCapture('examples\media\squat_front_ad_trim.mp4')
cap = cv2.VideoCapture('examples\media\mike_front_view_squat.mp4')
print("Number of frames in video: ",cap.get(cv2.CAP_PROP_FRAME_COUNT))
# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

cv2.namedWindow('Peach Factor',cv2.WINDOW_AUTOSIZE)
text = "Good Form!"

count = 0

while(True):
    
    ret, frame = cap.read()
    if ret == True:
        if count > 2:
              
            #right_temp_df = right_knee_df.iloc[count]
            #left_temp_df = left_knee_df.iloc[count] 
                
            #draw_point(frame, left_knee_df.iloc[count])
            draw_label(frame, text)
            #cv2.putText(frame,text,(150,500), font, 1,(255,255,255),2,cv2.LINE_AA)
            cv2.imshow('Peach Factor',frame)

            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
    else:
        break
    count+=1
     
cap.release()
cv2.destroyAllWindows()

Number of frames in video:  0.0
Error opening video stream or file


In [39]:
### Ok so next steps.
# 1. line sets.  Polygone line sets can be determined by creating different polygons each time the gradient changes
# 2. Lines can be filtered by length and removed from the plotting func.
# 3. Plotting should be adjusted such that the poly line plot call is made separately for each grouping of points

### Video with Knee Tracking Lines - Testing v1 up/down motion tracking

In [40]:
#cap = cv2.VideoCapture('examples\media\squat_front_ad_trim.mp4')
cap = cv2.VideoCapture('examples\media\mike_front_view_squat.mp4')
print("Number of frames in video: ",cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

cv2.namedWindow('Peach Factor',cv2.WINDOW_AUTOSIZE)
text = "This man needs help."
count = 0

while(True):
    
    ret, frame = cap.read()
    if ret == True:
        if count > 80:
            right_temp_df = right_knee_df.iloc[count-80:count]
            left_temp_df = left_knee_df.iloc[count-80:count]
            # Expected right knee, red line --> direct of motion is up
            draw_poly_line(frame, right_temp_df.loc[right_temp_df['gradient'] > 0].values, (255,0,0))
            # Expected right knee, blue line --> direct of motion is down
            draw_poly_line(frame, right_temp_df.loc[right_temp_df['gradient'] < 0].values, (0,0,255))
            
            # Expected left knee, red line --> direct of motion is up
            draw_poly_line(frame, left_temp_df.loc[left_temp_df['gradient'] > 0].values, (255,0,0))
            # Expected left knee, blue line --> direct of motion is down
            draw_poly_line(frame, left_temp_df.loc[left_temp_df['gradient'] < 0].values, (0,0,255))
            
            draw_label(frame, text)
            #cv2.putText(frame,text,(150,500), font, 1,(255,255,255),2,cv2.LINE_AA)
            cv2.imshow('Peach Factor',frame)

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break
    count+=1
     
cap.release()
cv2.destroyAllWindows()

Number of frames in video:  0.0
Error opening video stream or file


### Video with Knee Tracking Lines - All points

In [ ]:
#cap = cv2.VideoCapture('examples\media\squat_front_ad_trim.mp4')
cap = cv2.VideoCapture('examples\media\mike_front_view_squat.mp4')
print("Number of frames in video: ",cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

cv2.namedWindow('Peach Factor',cv2.WINDOW_AUTOSIZE)
text = "This man needs help."
count = 0

while(True):
    
    ret, frame = cap.read()
    if ret == True:
        if count > 2:

            draw_poly_line(frame, right_knee_df.iloc[0:count].values)
            draw_poly_line(frame, left_knee_df.iloc[0:count].values)
            draw_label(frame, text)
            #cv2.putText(frame,text,(150,500), font, 1,(255,255,255),2,cv2.LINE_AA)
            cv2.imshow('Peach Factor',frame)

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break
    count+=1
     
cap.release()
cv2.destroyAllWindows()

In [ ]:
#cap = cv2.VideoCapture('examples\media\squat_front_ad_trim.mp4')
cap = cv2.VideoCapture('examples\media\mike_front_view_squat.mp4')
print("Number of frames in video: ",cap.get(cv2.CAP_PROP_FRAME_COUNT))
# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

cv2.namedWindow('Peach Factor',cv2.WINDOW_AUTOSIZE)
text = "This man needs help."

count = 0

while(True):
    
    ret, frame = cap.read()
    if ret == True:
        
        for point in range(0,count):
            draw_point(frame, left_knee_df.iloc[point])
            draw_point(frame, right_knee_df.iloc[point])
        draw_label(frame, text)
        #cv2.putText(frame,text,(150,500), font, 1,(255,255,255),2,cv2.LINE_AA)
        cv2.imshow('Peach Factor',frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break
    count+=1
     
cap.release()
cv2.destroyAllWindows()

In [ ]:
print(frame.shape)


In [ ]:
# To Do
# 1. Save a sub set of all left knee, right knee and distance between the two
# 2. plot all the points, diff color for left and right
# 3. look for outliers
# find way to auto-classify
# 4. Plot knee points with line back onto video frame

In [ ]:
nasty_file = "squat_front_trim_000000000570_keypoints.json"
try:
    temp_df = pd.read_json(path_to_json+nasty_file, orient='record')
    temp_df = pd.DataFrame.from_dict(temp_df.values[0][0], orient='index')
except:
    print('bad record')
temp_df

In [ ]:
 cv2.polylines(img,[pts],True,(0,255,255))

In [ ]:
blue = (255, 0, 0)
red = (0, 0, 255)
green = (0, 255, 0)
violet = (180, 0, 180)
yellow = (0, 180, 180)
white = (255, 255, 255)

In [ ]:
cv2.line(image, (50, 30), (450, 35), blue, thickness=5)
cv2.circle(image, (240, 205), 23, red, -1)
cv2.rectangle(image, (50, 60), (450, 95), green, -1)
cv2.ellipse(image, (250, 150), (80, 20), 5, 0, 360, violet, -1)
points = np.array([[[140, 230], [380, 230], [320, 250], [250, 280]]], np.int32)
cv2.polylines(image, [points], True, yellow, thickness=3)

In [ ]:
font_scale = 1.5
font = cv2.FONT_HERSHEY_PLAIN

# set the rectangle background to white
rectangle_bgr = (255, 255, 255)
# make a black image
img = np.zeros((500, 500, 3))
# set some text
text = "Some text in a box!"
# get the width and height of the text box
(text_width, text_height) = cv2.getTextSize(text, font, fontScale=font_scale, thickness=1)[0]
# set the text start position
text_offset_x = 10
text_offset_y = img.shape[0] - 25
# make the coords of the box with a small padding of two pixels
box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width - 2, text_offset_y - text_height - 2))
cv2.rectangle(img, box_coords[0], box_coords[1], rectangle_bgr, cv2.FILLED)
cv2.putText(img, text, (text_offset_x, text_offset_y), font, fontScale=font_scale, color=(0, 0, 0), thickness=1)
cv2.imshow("A box!", img)
cv2.waitKey(0)

In [ ]:
print('Size of img: ', img.shape)

In [ ]:
# Right 9, 10, 11, 22
# Left 12, 13, 14, 19

In [ ]:

'''
// Result for BODY_25 (25 body parts consisting of COCO + foot)
// const std::map<unsigned int, std::string> POSE_BODY_25_BODY_PARTS {
//     {0,  "Nose"},
//     {1,  "Neck"},
//     {2,  "RShoulder"},
//     {3,  "RElbow"},
//     {4,  "RWrist"},
//     {5,  "LShoulder"},
//     {6,  "LElbow"},
//     {7,  "LWrist"},
//     {8,  "MidHip"},
//     {9,  "RHip"},
//     {10, "RKnee"},
//     {11, "RAnkle"},
//     {12, "LHip"},
//     {13, "LKnee"},
//     {14, "LAnkle"},
//     {15, "REye"},
//     {16, "LEye"},
//     {17, "REar"},
//     {18, "LEar"},
//     {19, "LBigToe"},
//     {20, "LSmallToe"},
//     {21, "LHeel"},
//     {22, "RBigToe"},
//     {23, "RSmallToe"},
//     {24, "RHeel"},
//     {25, "Background"}
// };

'''

In [ ]:

'''
multi-select technique
df.loc[(df["B"] > 50) & (df["C"] == 900), "A"] *= 1000
df
      A   B    C
0     9  40  300
1     9  70  700
2  5000  70  900
3  8000  80  900
4     7  50  200
5     9  30  900
'''

import cv2
 
img = cv2.imread('lena.png')
 
# (1) create a copy of the original:
overlay = img.copy()
# (2) draw shapes:
cv2.circle(overlay, (133, 132), 12, (0, 255, 0), -1)
cv2.circle(overlay, (166, 132), 12, (0, 255, 0), -1)
# (3) blend with the original:
opacity = 0.4
cv2.addWeighted(overlay, opacity, img, 1 - opacity, 0, img)
 
# display result (press 'q' to quit):
cv2.namedWindow('Transparency')
cv2.imshow('Transparency', img)
while (cv2.waitKey() & 0xff) != ord('q'): pass
cv2.destroyAllWindows()